In [71]:
import os
import requests

video_path = "data/example/input_video.mp4"
model = "yolo"
interesting_frame = 210

images_for_paper_path = "data/example/images_for_paper"

In [72]:
# Test if service is running
response = requests.get("http://localhost:4321/")
print(response.json())

{'message': 'Football Referee Evaluation API is running'}


In [73]:
def make_post_request(endpoint: str, body: dict) -> dict:
    response = requests.post(f"http://localhost:4321/{endpoint}", json=body)
    if response.status_code != 200:
        msg = f"Request failed with status code {response.status_code}: {response.text}"
        raise Exception(msg)
    return response.json()

In [48]:
# Track players
tracking_results_path = "data/example/predictions/tracking_results"
tracking_results = make_post_request(
    endpoint="track", 
    body = {
        "video_path": video_path,
        "model": model,
        "results_path": tracking_results_path
    }
)

In [74]:
# Load the detections file
import json
with open(os.path.join(tracking_results_path, "detections.json"), "r") as f:
    detections = json.load(f)

In [75]:
# Load the video and extract the interesting frame
import cv2
import numpy as np
import os

# Load the video
cap = cv2.VideoCapture(video_path)

# Set the frame position to the interesting frame
cap.set(cv2.CAP_PROP_POS_FRAMES, interesting_frame)

# Read the frame
ret, frame = cap.read()
if not ret:
    raise Exception(f"Could not read frame {interesting_frame}")

# Get detections for this specific frame
frame_detections = next((item for item in detections if item["frame_id"] == interesting_frame), None)

if frame_detections is None:
    print(f"No detections found for frame {interesting_frame}")    

# Create a copy of the frame to draw on
annotated_frame = frame.copy()

# Draw bounding boxes and IDs
for detection in frame_detections["detections"]:
    # Extract bbox coordinates
    bbox = detection["bbox"]
    x1, y1, x2, y2 = bbox["x1"], bbox["y1"], bbox["x2"], bbox["y2"]
    track_id = detection["track_id"]
    
    # Draw bright green bbox
    color = (0, 255, 0)  # Bright green in BGR
    thickness = 2
    cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
    
    # Add track ID text
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(annotated_frame, f"{track_id}", (int(x1), int(y1) - 10), 
                font, 0.7, color, 2)
    
    # Draw a black circle at the player's feet (bottom center of bounding box)
    feet_x = int((x1 + x2) / 2)  # Center x-coordinate
    feet_y = int(y2)  # Bottom y-coordinate
    circle_color = (0, 0, 0)  # Black in BGR
    circle_radius = 12
    circle_thickness = -1  # Filled circle
    cv2.circle(annotated_frame, (feet_x, feet_y), circle_radius, circle_color, circle_thickness)

# Save the annotated frame as a PNG image
os.makedirs(images_for_paper_path, exist_ok=True)
output_path = f"{images_for_paper_path}/tracking.png"
cv2.imwrite(output_path, annotated_frame)

print(f"Saved annotated frame to {output_path}")

# Release the video capture
cap.release()

Saved annotated frame to data/example/images_for_paper/tracking.png


In [56]:
# Track players
transform_coordinates_results_path = "data/example/predictions/coordinate_transformation"
transform_coordinates_results = make_post_request(
    endpoint="transform-coordinates", 
    body = {
        "video_path": video_path,
        "detections": detections,
        "results_path": transform_coordinates_results_path
    }
)

In [78]:
# Load the detections file
import json
with open(os.path.join(transform_coordinates_results_path, "detections.json"), "r") as f:
    detections = json.load(f)
    
warped_image_path = f"data/example/predictions/coordinate_transformation/warped_images/frame_{interesting_frame:06d}.jpg"

In [79]:
# Load the warped image and the pitch background
warped_image = cv2.imread(warped_image_path)
pitch_background = cv2.imread("src/utils/pitch_2.png")

if warped_image is None:
    print(f"Error: Could not load warped image from {warped_image_path}")
elif pitch_background is None:
    print(f"Error: Could not load pitch background from pitch_2.png")
else:
    # Get pitch background dimensions
    pitch_height, pitch_width = pitch_background.shape[:2]
    
    # Resize warped image to match pitch background dimensions
    resized_warped_image = cv2.resize(warped_image, (pitch_width, pitch_height))
    
    # Create a semi-transparent overlay
    alpha = 0.6  # Transparency factor (0.0 = fully transparent, 1.0 = fully opaque)
    overlay = pitch_background.copy()
    
    # Overlay the warped image on the pitch background
    cv2.addWeighted(resized_warped_image, alpha, pitch_background, 1 - alpha, 0, overlay)
    
    # Create a copy for annotation
    annotated_image = overlay.copy()
    
    # Find the detections for the interesting frame
    frame_detections = None
    for frame_data in detections:
        if frame_data["frame_id"] == interesting_frame:
            frame_detections = frame_data["detections"]
            break
    
    if frame_detections:
        # Draw black dots at player positions
        for detection in frame_detections:
            if "minimap_coordinates" in detection and detection["minimap_coordinates"]:
                coords = detection["minimap_coordinates"]
                
                # Rescale coordinates using x_max and y_max to match pitch dimensions
                x_max = coords["x_max"]
                y_max = coords["y_max"]
                x_scaled = int((coords["x"] / x_max) * pitch_width)
                y_scaled = int((coords["y"] / y_max) * pitch_height)
                
                # Draw a filled black circle (dot)
                cv2.circle(annotated_image, (x_scaled, y_scaled), 10, (0, 0, 0), -1)  # -1 means filled circle
                
    # Save the annotated image
    warped_output_path = f"{images_for_paper_path}/warped_tracking.png"
    cv2.imwrite(warped_output_path, annotated_image)
    print(f"Saved annotated warped image to {warped_output_path}")


Saved annotated warped image to data/example/images_for_paper/warped_tracking.png


In [80]:
# Assign colors to players
color_assignment_results_path = "data/example/predictions/color_assignment"
color_assignment_results = make_post_request(
    endpoint="assign-colors", 
    body = {
        "video_path": video_path,
        "detections": detections,
        "results_path": color_assignment_results_path
    }
)

In [81]:
with open(os.path.join(color_assignment_results_path, "detections.json"), "r") as f:
    detections = json.load(f)

In [135]:
# Assign roles
role_assignment_results_path = "data/example/predictions/role_assignment"
role_assignment_results = make_post_request(
    endpoint="assign-roles", 
    body = {
        "video_path": video_path,
        "detections": detections,
        "results_path": role_assignment_results_path
    }
)